# Calling OpenAI API

In [11]:
import os
from dotenv import load_dotenv
from openai import OpenAI

In [12]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [13]:
client = OpenAI(
  api_key=OPENAI_API_KEY
)

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  store=True,
  messages=[
    {"role": "user", "content": "How can i use .env files with python example"}
  ]
)

print(completion.choices[0].message.content);

Using `.env` files in Python applications is a common practice for managing environment variables and configuration settings without hardcoding them in your source code. The most popular way to do this is by using the `python-dotenv` library, which simplifies the process of loading environment variables from a `.env` file.

Here's a step-by-step guide along with an example:

### Step 1: Install `python-dotenv`

If you don't have `python-dotenv` installed yet, you can install it using pip:

```bash
pip install python-dotenv
```

### Step 2: Create a `.env` File

Create a file named `.env` in your project directory. This file should contain your environment variables in the format `KEY=VALUE`. For example:

```plaintext
# .env
API_KEY=your_api_key_here
DATABASE_URL=sqlite:///my_database.db
DEBUG=True
```

### Step 3: Load the Environment Variables in Your Python Code

You can now load the environment variables from the `.env` file in your Python script. Here’s an example:

```python
impo

# Retrieval

In [21]:
# search-engine in the search-engine folder installed via uv add.
import minsearch
import json

In [22]:
documents = []

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [ ]:
# creating index
index = minsearch.Index(
            text_fields=["qeustion", "text", "section"],
            keyword_fields=["course"]
)

index.fit(documents)

# Performing a search
q="the course has already started, can i still enroll?"

boost = {"question": 3.0,
         "section": 0.5,
        }

results = index.search(
    query=q,
    filter_dict= {'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
results